Copyright (c) Microsoft Corporation. All rights reserved. Licensed under the MIT License.

Vrijwel alle code in deze workshop is gebaseerd op de Azure Machine Learning voorbeelden van Microsoft.

# Ordina mTech AI Workshop: Train een classificatie model voor afbeeldingen middels Azure Machine Learning

In deze workshop ga je een machine learning model trainen op resources die in Azure draaien. We gaan stapsgewijs het model trainen en deployen middels dit Azure (Jupyter) notebook.

Qua model wat we gaan trainen hebben we het redelijk eenvoudig gehouden. We hebben overwogen en geprobeerd andere datasets te gebruiken, maar het trainen en optuigen van het model duurt dan simpelweg te lang voor een workshop. Wil je hier verder mee, geef dan een gil aan John, Martijn of Roelant voor een extra dataset waar je in de avonduren op los kan gaan.

We gaan een 'simpel' logistisch regressie model bouwen voor de [MNIST](http://yann.lecun.com/exdb/mnist/) dataset gebruik makende van Azure Machine Learning services. MNIST is een zeer bekende dataset bestaande uit 70.000 zwart/wit plaatjes. Elk plaatje is een afbeelding van een handgeschreven getal tussen de 0 en 9 van 28x28 pixels. Het model wordt een multi-class classificatie model wat van een plaatje van een handgeschreven getal gaat inschatten welk getal tussen 0 en 9 dit is.

## Prerequisites

- Laptop
- Azure subscription
- Github account

(Maar als je dit kan lezen, ben je voor alledrie geslaagd)

## Intro

Voor de mensen die nog nooit met een Azure notebook of Jupyter notebook hebben gewerkt, kan dit in het begin een beetje gek voelen. Kort gezegd is een Azure/Jupyter notebook een verzameling van tekst en code, waarbij je de code direct op de pagina kan uitvoeren. Verder ontkom je niet aan python als je iets met machine learning wil doen. Voordeel van python is dat het extreem goed leesbaar is.

Naast de code segmenten staat een 'In [ ]' indicator. Een notebook houdt state en historie bij, dus als je de volgende keer terugkeert in je notebook, is de staat zoals je die hebt achtergelaten. De 'In [ ]' indicator is leeg, mits de stap niet eerder is uitgevoerd. Na uitvoeren van de code, wordt de volgorde van uitgevoerde stappen hier in bijgehouden. Deze index wordt elke keer opgehoogd als de stap wordt uitgevoerd. Na 1x uitvoeren staat er dus 'In [1]'. Als de stap nog wordt uitgevoerd staat er een asterisk (een sterretje dus) 'In [\*]'. Je kunt eventuele volgende stappen wel in de queue zetten, maar deze worden pas uitgevoerd als de lopende stap klaar is.

## Configuratie

Het Azure notebook heeft enige configuratie nodig. Het gaat daarbij om 4 variabelen:<br>
**SUBSCRIPTION_ID** = je Azure subscription Id. Je vindt deze in de portal, bijv. onder Cost Management of onder Subscriptions. <br>

De onderstaande waarden kan je aanpassen, maar kan je ook zo laten staan.<br>
**RESOURCE_GROUP** = de naam van een resource group. Als deze niet bestaat, wordt deze aangemaakt. <br>
**WORKSPACE_NAME** = de naam van de op te spinnen workspace. Als deze niet bestaat, wordt deze aangemaakt. <br>
**WORKSPACE_REGION** = westeurope (Of als je dapper wil zijn, een andere) 

In [ ]:
import os

subscription_id = os.getenv("SUBSCRIPTION_ID", default="[SUBSCRIPTION_ID_HERE]")
resource_group = os.getenv("RESOURCE_GROUP", default="Ordina-ML-rg")
workspace_name = os.getenv("WORKSPACE_NAME", default="Ordina-ML-service")
workspace_region = os.getenv("WORKSPACE_REGION", default="westeurope")

### Workspace
We gaan eerst controleren of je workspace al bestaat. Voer de code hieronder uit. De code probeert een workspace op te vragen en te exporteren naar een config bestand. Als de workspace al bestaat, ben je daarna klaar met het configuratie deel en kan je door naar het [importeren van de python packages](#Setup-van-de-development-omgeving-van-je-notebook).

In [ ]:
from azureml.core import Workspace

try:
    ws = Workspace(subscription_id = subscription_id, resource_group = resource_group, workspace_name = workspace_name)
    # write the details of the workspace to a configuration file to the notebook library
    ws.write_config()
    print("Workspace configuration succeeded. Skip the workspace creation steps below")
except:
    print("Workspace not accessible. Change your parameters or create a new workspace below")

Bij het uitvoeren van bovenstaande code, kan het zijn dat je meldingen krijgt over authenticatie / credentials. Dit is verwacht en is een kwestie van gewoon doen wat er staat. Het is het (tijdelijk) toegang geven van het Azure notebook op jouw Azure resources.

We gaan nu een nieuwe Azure ML workspace opspinnen:

In [ ]:
from azureml.core import Workspace

# Create the workspace using the specified parameters
ws = Workspace.create(name = workspace_name,
                      subscription_id = subscription_id,
                      resource_group = resource_group, 
                      location = workspace_region,
                      create_resource_group = True,
                      exist_ok = True)
ws.get_details()

# write the details of the workspace to a configuration file to the notebook library
ws.write_config()

Ook hier weer een rode balk als je workspace, maar deze zou informational moeten zijn en je wijzen op het feit dat een resource groep is aangemaakt, omdat deze nog niet bestond. Het eind resultaat zou een melding moeten zijn dat je config succesvol is weggeschreven. Als je resource groep wel al bestaat, krijg je uiteraard geen melding.

We hebben straks ook nog compute resources nodig. Hoewel machine learning modellen uiteraard het beste trainen op GPUs (**<Commercial_Break>** voor als je je afvraagt waarom dat is, Roelant gaat dat proberen uit te leggen tijdens zijn sessie op FutureTech as. woensdag **</Commercial_Break>**) gaan we geen GPU cluster opspinnen. In het originele Microsoft voorbeeld staat de code, dus als je je dapper voelt, voel je vrij, maar in de standaard flow gaan we uit van een CPU cluster waarop we ons model gaan trainen.

Voer nu onderstaande code uit. Het creëren van het de compute resources kan even duren, dus voel je vrij om anderen te helpen tot deze stap te komen, haal een bak koffie of bespreek je favoriete Netflix serie met iemand die ook net zo ver is.

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Choose a name for your CPU cluster
cpu_cluster_name = "cpucluster"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print("Found existing cpucluster")
except ComputeTargetException:
    print("Creating new cpucluster")
    
    # Specify the configuration for the new cluster
    compute_config = AmlCompute.provisioning_configuration(vm_size="STANDARD_D2_V2",
                                                           min_nodes=1,
                                                           max_nodes=4)

    # Create the cluster with the specified name and configuration
    compute_target = ComputeTarget.create(ws, cpu_cluster_name, compute_config)
    
    # Wait for the cluster to complete, show the output log
    compute_target.wait_for_completion(show_output=True)

Bovenstaande code geeft feedback over de voortgang van opspinnen van het cluster. We zetten het minimaal aantal draaiende nodes op 1, zodat we daar straks minder lang op hoeven te wachten. Het opspinnen van nieuwe nodes op een 'koude' workspace is vaak een koffie-haal-momentje. Nu duurt het ongeveer 2-3 minuten. 

We gaan nu door met het installeren van pre requisites binnen je Azure notebook.

## Setup van de development omgeving van je notebook

### Importeren packages

Importeer met de volgende stap de Python packages die je nodig hebt. Als validatie wordt hier de Azure ML SDK versie als output gegeven. Deze doet er voor de rest van de workshop niet echt toe, maar voor de toekomst kan het handig zijn om problemen met het notebook te detecteren.

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

import azureml.core
from azureml.core import Workspace

# check core SDK version number
print("Azure ML SDK Version: ", azureml.core.VERSION)

### Connect naar de workspace

We gaan een connectie maken naar de zojuist gecreërde workspace. `Workspace.from_config()` leest de inhoud van het bestand **config.json** en laadt alles in een object `ws`. Dit object zullen we verderop in het notebook gebruiken.

In [ ]:
ws = Workspace.from_config()
print(ws.name, ws.location, ws.resource_group, ws.location, sep='\t')

### Maak een experiment

Maak een experiment aan in je workspace om de voortgang van het trainen van het model te volgen. Zie het als een container waarin je model voortgang te zien is. Een workspace kan meerdere experimenten bevatten, maar we gebruiken er nu maar één ('sklearn-mnist'): 

In [ ]:
experiment_name = 'sklearn-mnist'

from azureml.core import Experiment
exp = Experiment(workspace=ws, name=experiment_name)

We kunnen nu daadwerkelijk je model te gaan trainen. Neem even een moment om stil te staan bij wat we net gedaan hebben. Voor je gevoel heb je een paar keer op run geklikt, maar je hebt gewoon een heel platform gebouwd waarop je machine learning modellen kan trainen en een compute cluster opgespind met bad-ass machines die in de cloud jouw modellen kunnen doorrekenen. Je kan nu al trots zijn!

## MNIST dataset

Voordat we een model kunnen trainen, is het handig als je begrijpt welke data er in de dataset zit en wat het doel is van deze workshop.

We gaan nu:

* De MNIST dataset downloaden
* Waat voorbeelden bekijken van plaatjes (van handgeschreven cijfers)
* De data Azure in laden

### Download de MNIST dataset

Onderstaand code blok gaat de MNIST dataset downloaden en de bestanden naar een `data` map binnen het notebook wegschrijven. Het gaat hierbij om zowel de plaatjes alsook de labels voor testen en trainen van het model.

In [ ]:
import urllib.request

data_folder = os.path.join(os.getcwd(), 'data')
os.makedirs(data_folder, exist_ok=True)

urllib.request.urlretrieve('http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz', filename=os.path.join(data_folder, 'train-images.gz'))
urllib.request.urlretrieve('http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz', filename=os.path.join(data_folder, 'train-labels.gz'))
urllib.request.urlretrieve('http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz', filename=os.path.join(data_folder, 'test-images.gz'))
urllib.request.urlretrieve('http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz', filename=os.path.join(data_folder, 'test-labels.gz'))

### Voorbeelden dataset bekijken

De zips worden in onderstaand code blok in `numpy` arrays geladen. Een `numpy` array is een geïndexeerde matrix van waarden van allemaal hetzelfde type. We gebruiken vervolgens de plotting library `matplotlib` om 30 willekeurige plaatjes uit de dataset te tonen inclusief hun label. 

In [ ]:
from utils import load_data

# note we also shrink the intensity values (X) from 0-255 to 0-1. This helps the model converge faster.
X_train = load_data(os.path.join(data_folder, 'train-images.gz'), False) / 255.0
X_test = load_data(os.path.join(data_folder, 'test-images.gz'), False) / 255.0
y_train = load_data(os.path.join(data_folder, 'train-labels.gz'), True).reshape(-1)
y_test = load_data(os.path.join(data_folder, 'test-labels.gz'), True).reshape(-1)

# now let's show some randomly chosen images from the traininng set.
count = 0
sample_size = 30
plt.figure(figsize = (16, 6))
for i in np.random.permutation(X_train.shape[0])[:sample_size]:
    count = count + 1
    plt.subplot(1, sample_size, count)
    plt.axhline('')
    plt.axvline('')
    plt.text(x=10, y=-10, s=y_train[i], fontsize=18)
    plt.imshow(X_train[i].reshape(28, 28), cmap=plt.cm.Greys)
plt.show()

De voorbeeld plaatjes geven je een idee van de inhoud van de dataset en welk model we gaan trainen. Het idee is dus dat het model straks aan de hand van het plaatje kan gaan voorspellen welk cijfer het is.

### Upload data naar azure

Aangezien we in de cloud het model gaan trainen, zal de data ook naar de cloud moeten worden opgeladen zodat de Azure ML workspace hier toegang toe heeft. De datastore in de workspace is hier uitermate geschikt voor. Je kan hier data naar opladen en de compute resources hebben er standaard al toegang toe. Onder water zit hier 'gewoon' een blob storage onder.

Onderstaand code blok laadt de MNIST dataset bestanden in een folder `mnist` in de root van je datastore binnen je Azure ML workspace.

In [ ]:
ds = ws.get_default_datastore()
print(ds.datastore_type, ds.account_name, ds.container_name)

ds.upload(src_dir=data_folder, target_path='mnist', overwrite=True, show_progress=True)

## Train het model in de cloud

In een van de voorgaande stappen hebben we een compute cluster opgespind. Op dit cluster gaan we nu een training job starten.

### Folder notebook maken
We gaan eerst een folder aanmaken in het notebook die we zullen vullen met code die we naar het compute cluster zullen gaan sturen.

In [ ]:
import os
script_folder = os.path.join(os.getcwd(), "sklearn-mnist")
os.makedirs(script_folder, exist_ok=True)

### Trainingscript maken

Het trainingscript is waar alle 'magie' gebeurt. Dit is het script wat:
- de compute nodes binnen het cluster straks uit gaan voeren
- de train en test data inlaadt
- een `LogisticRegression` algoritme gebruikt om het model te trainen (`clf.predict`)
- het model exporteert

Dit is dus het stuk waar feitelijk machine learning wordt toegepast. Dit model gebruikt dit specifieke algoritme omdat het sneller te trainen is dan een neuraal netwerk met toch een vergelijkbare nauwkeurigheid. De `regularization rate` waar over gesproken wordt in de comments is om `overfitting` van het model te voorkomen.

Met onderstaand code blok wordt er een file `train.py` weggeschreven in de eerder aangemaakte folder.

In [ ]:
%%writefile $script_folder/train.py

import argparse
import os
import numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn.externals import joblib

from azureml.core import Run
from utils import load_data

# let user feed in 2 parameters, the location of the data files (from datastore), and the regularization rate of the logistic regression model
parser = argparse.ArgumentParser()
parser.add_argument('--data-folder', type=str, dest='data_folder', help='data folder mounting point')
parser.add_argument('--regularization', type=float, dest='reg', default=0.01, help='regularization rate')
args = parser.parse_args()

data_folder = args.data_folder
print('Data folder:', data_folder)

# load train and test set into numpy arrays
# note we scale the pixel intensity values to 0-1 (by dividing it with 255.0) so the model can converge faster.
X_train = load_data(os.path.join(data_folder, 'train-images.gz'), False) / 255.0
X_test = load_data(os.path.join(data_folder, 'test-images.gz'), False) / 255.0
y_train = load_data(os.path.join(data_folder, 'train-labels.gz'), True).reshape(-1)
y_test = load_data(os.path.join(data_folder, 'test-labels.gz'), True).reshape(-1)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape, sep = '\n')

# get hold of the current run
run = Run.get_context()

print('Train a logistic regression model with regularization rate of', args.reg)
clf = LogisticRegression(C=1.0/args.reg, random_state=42)
clf.fit(X_train, y_train)

print('Predict the test set')
y_hat = clf.predict(X_test)

# calculate accuracy on the prediction
acc = np.average(y_hat == y_test)
print('Accuracy is', acc)

run.log('regularization rate', np.float(args.reg))
run.log('accuracy', np.float(acc))

os.makedirs('outputs', exist_ok=True)
# note file saved in the outputs folder is automatically uploaded into experiment record
joblib.dump(value=clf, filename='outputs/sklearn_mnist_model.pkl')

De `joblib.dump` schrijft het model weg naar de outputs folder. Deze wordt automatisch opgeladen naar het experiment in je Azure ML workspace. Dit gebeurt dus straks binnen het compute cluster als het script klaar is met trainen op het cluster en de output gaat terugschrijven naar het experiment.

Onderstaande code blok kopieert `utils.py` naar de script folder. Dit pyton bestand bevat helpers om de dataset in te laden en moet dus aanwezig zijn voor het compute cluster.

In [ ]:
import shutil
shutil.copy('utils.py', script_folder)

### Estimator maken

Een 'estimator' is het object wat het daadwerkelijke werk doet op de compute nodes. Deze heeft nodig:

* De scripts en inhoud van de `sklearn-mnist`-folder. 
* De compute target
* De naam van het trainingsscrip: `train.py`
* De training parameters (script_params) 
* De python packages die nodig zijn voor het trainen van het model

De data_folder wijst hier naar de datastore in je Azure ML Workspace (`ds.path('mnist').as_mount()`).

In [ ]:
from azureml.train.estimator import Estimator

script_params = {
    '--data-folder': ds.path('mnist').as_mount(),
    '--regularization': 0.05
}

est = Estimator(source_directory=script_folder,
                script_params=script_params,
                compute_target=compute_target,
                entry_script='train.py',
                conda_packages=['scikit-learn'])

### Job naar remote cluster sturen

We gaan onze eerste job in ons experiment starten door gebruik te maken van de estimator. Eventueel kan je in de Azure portal de voortgang bijhouden van het trainen van het model. In dit geval is de training job normaal gezien relatief snel klaar, maar voor ingewikkeldere modellen is het een goede manier om de voortgang te zien.

Voor nu onderstaand code blok uit om het experiment te starten, maar lees vooral wel verder.

In [ ]:
run = exp.submit(config=est)
run

(Call is async, dus vertrouw hier niet direct de return status)

## Job monitoren op het cluster

De eerste run van het model kan ook weer even duren en hangt ook af hoe snel Azure nodes wil opspinnen in je cluster, dus wellicht een goed moment om weer even collega's te polsen over hun voortgang, weer een rondje koffie te doen, of wellicht ben je een aantal termen en concepten tegengekomen die je even op wil zoeken op Google. Vervolg runs van het model zijn vrijwel altijd sneller, omdat er een docker container wordt gemaakt met jouw estimator data die later kan worden hergebruikt.

Om je een beeld te vormen wat er gebeurt terwijl je wacht / koffie haalt:

- **Docker image aanmaken**: Er wordt een docker image gemaakt van je estimator die naar ACR wordt gestuurd. Weet je niet wat ACR is? Nu is het een mooi moment om dit op te zoeken.
- **Cluster nodes opschalen**: We hebben het compute cluster aangemaakt zonder draaiende nodes. Dit is goedkoper, maar het opspinnen van machines van deze orde van grootte kost nou eenmaal even tijd
- **Trainen van het model**: Hier gebeurt het 'echte' werk. De scripts worden naar de compute target gestuurd, de data store wordt in je workspace gehangen en het trainingsscript wordt uitgevoerd. 
- **Post-Processing**: Het resultaat wordt teruggeschreven naar je Azure ML workspace

...mocht je het toch lang vinden duren... realiseer dan ook hoeveel tijd dit allemaal kosten voordat we Azure ML workspaces hadden. Er zat altijd een berg handmatig werk in om alle stappen te doorlopen en vaak zat er niks anders op dan te trainen op een data science VM in Azure die je helemaal moest inrichten, of nog erger dat je moest trainen op je lokale machine.

### Jupyter widget

Wellicht was je reeds onder de indruk van Jupyter notebooks, maar notebooks kunnen ook widgets bevatten. Hieronder een widget die elke 10-15 seconden ververst en de voortgang toont van je job die draait binnen je experiment.

In [ ]:
from azureml.widgets import RunDetails
RunDetails(run).show()

### Is het al klaar?

Naast de widget en het monitoren in de portal kan je ook 'gewoon' wachten tot het trainen van het model klaar is.

Onderstaand code blok blijft in de 'running' state zolang je model nog met de trainingsrun bezig is.

In [ ]:
# specify show_output to True for a verbose log
run.wait_for_completion(show_output=False) 

### KLAAR! Toon nu het resultaat

Gefeliciteerd! Je hebt zojuist je eerste machine learning model getraind. Wees nog trotser dan eerst op jezelf. Misschien voelt het nog niet anders, maar er is een tijd voor en een tijd na het trainen van je eerste ML model.

We zijn uiteraard wel benieuwd naar de performance en nauwkeurigheid van voorspellen van je zojuist getrainde model:

In [ ]:
print(run.get_metrics())

## Registreer model

Doordat het trainingsscript het model wegschrijft naar de `outputs` folder, wordt het automagisch ge-upload naar het experiment in je workspace. 

Registreer nu het model in de Azure ML workspace zodat je dit model later kan bevragen, onderzoeken of deployen.

In [ ]:
# register model 
model = run.register_model(model_name='sklearn_mnist', model_path='outputs/sklearn_mnist_model.pkl')
print(model.name, model.id, model.version, sep='\t')

## Download model

We gaan het model nu downloaden naar ons notebook:

In [ ]:
model.download(target_dir=os.getcwd(), exist_ok=True)

# verify the downloaded model file
file_path = os.path.join(os.getcwd(), "sklearn_mnist_model.pkl")

os.stat(file_path)

## Model lokaal testen

Voordat we het model gaan deployen, willen we het eerst lokaal testen.

### Laad test data

Laad de test data uit de **./data/** folder uit je notebook. In een eerdere stap hebben we hier de gedownloade dataset neergezet.

In [ ]:
from utils import load_data
import os

data_folder = os.path.join(os.getcwd(), 'data')
# note we also shrink the intensity values (X) from 0-255 to 0-1. This helps the neural network converge faster
X_test = load_data(os.path.join(data_folder, 'test-images.gz'), False) / 255.0
y_test = load_data(os.path.join(data_folder, 'test-labels.gz'), True).reshape(-1)

### Test data scoren met model

We gaan nu de test dataset door het model heen halen:

In [ ]:
import pickle
from sklearn.externals import joblib

clf = joblib.load( os.path.join(os.getcwd(), 'sklearn_mnist_model.pkl'))
y_hat = clf.predict(X_test)

Je krijgt hier overigens weer een rode waarschuwing over versionering van de LogisticRegression. We nemen de waarschuwing ter kennisgeving aan. Dit is soms het nadeel van de notebooks en het niet op detail managen van alle dependencies, maar dat kan je ook net zo hard uitleggen als voordeel. In dit geval schaadt het ons getrainde model niet.

### De confusion matrix

Nee... dit is niet een plek waar Neo de weg niet weet. De naam van een confusion matrix is het ingewikkelste eraan. Het is een matrix (Excel is een 2D matrix = een grid) met alle verwachte uitkomsten uitgezet tegen de werkelijke uitkomsten. Idealiter heb je voor elke voorspelde waarde ook de juiste waarde hiervan uit het model teruggekregen. Boven de kolommen en voor de rijen moet je dus de getallen 0-9 denken. 

In [ ]:
from sklearn.metrics import confusion_matrix

conf_mx = confusion_matrix(y_test, y_hat)
print(conf_mx)
print('Overall accuracy:', np.average(y_hat == y_test))

Om het nog meer confusing te maken, kunnen we deze matrix ook visualiseren met `matplotlib`.  De ideale lijn van de visualisatie is dus van linksboven tot rechtsonder. Op deze plekken willen we goede voorspellingen (= hoge zekerheid = donkere vlakken) zien. Hoe hoger de foutmarge, des te lichter de cel is gekleurd. 

In [ ]:
# normalize the diagonal cells so that they don't overpower the rest of the cells when visualized
row_sums = conf_mx.sum(axis=1, keepdims=True)
norm_conf_mx = conf_mx / row_sums
np.fill_diagonal(norm_conf_mx, 0)

fig = plt.figure(figsize=(8,5))
ax = fig.add_subplot(111)
cax = ax.matshow(norm_conf_mx, cmap=plt.cm.bone)
ticks = np.arange(0, 10, 1)
ax.set_xticks(ticks)
ax.set_yticks(ticks)
ax.set_xticklabels(ticks)
ax.set_yticklabels(ticks)
fig.colorbar(cax)
plt.ylabel('true labels', fontsize=14)
plt.xlabel('predicted values', fontsize=14)
plt.savefig('conf.png')
plt.show()

Je ziet dus dat het systeem vaak een 3 voorspelt als het in werkelijkheid een 5 is en ook hier zie je weer de diagonale lijn van linksboven naar rechtsonder.

## Deployment

Nu we een werkend en OK scorend model hebben, willen we deze ook gaan consumeren in onze applicaties. We gaan dit doen middels een webservice die we in een Azure Container Instance (ACI) gaan laden. We hebben hiervoor het volgende nodig:
- Een scoring script wat het model inlaadt en gebruikt om voorspellingen (`model.predict`) uit te voeren 
- Een package dependencies file om de packages die nodig zijn in te laden
- Een ACI config file
- Het model

Per conventie staan er in het scoring script twee functies die worden aangeroepen door de web service: `init` en `run`

### Scoring script
Onderstaand code blok maakt het scoring script aan

In [ ]:
%%writefile score.py
import json
import numpy as np
import os
import pickle
from sklearn.externals import joblib
from sklearn.linear_model import LogisticRegression

from azureml.core.model import Model

def init():
    global model
    # retrieve the path to the model file using the model name
    model_path = Model.get_model_path('sklearn_mnist')
    model = joblib.load(model_path)

def run(raw_data):
    data = np.array(json.loads(raw_data)['data'])
    # make prediction
    y_hat = model.predict(data)
    # you can return any data type as long as it is JSON-serializable
    return y_hat.tolist()

### Package dependencies file

We schrijven hier de yaml weg met dependencies. Dit is enkel de `scikit-learn` library.

In [ ]:
from azureml.core.conda_dependencies import CondaDependencies 

myenv = CondaDependencies()
myenv.add_conda_package("scikit-learn")

with open("myenv.yml","w") as f:
    f.write(myenv.serialize_to_string())

...mocht je nieuwsgierig zijn hoe zo'n yaml er uitziet, maar voel je ook vrij dit code blok te skippen:

In [ ]:
with open("myenv.yml","r") as f:
    print(f.read())

### ACI configuratie bestand

We moeten ACI laten weten welke resources (CPU & RAM) er aangewend mogen worden voor het runnen van de container met het ML model. Het hangt uiteraard geheel af van je model welke resources je hiervoor nodig zal hebben, maar standaard is dat 1 core met 1 Gb en dit is doorgaans voldoende voor de meeste modellen. Je kan uiteraard in een later stadium altijd opschalen:

In [ ]:
from azureml.core.webservice import AciWebservice

aciconfig = AciWebservice.deploy_configuration(cpu_cores=1, 
                                               memory_gb=1, 
                                               tags={"data": "MNIST",  "method" : "sklearn"}, 
                                               description='Predict MNIST with sklearn')

### Deployment

Het model hadden we al, dus we kunnen nu daadwerkelijk gaan deployen naar ACI. Ook hier weer even een pas op de plaats en weer een moment om collega's te helpen, het weer te bespreken, maar kijk uit met koffie halen, want je hebt al twee bakken op inmiddels. Het deployen van een container naar ACI kan zomaar 7 minuten duren. 

In [ ]:
%%time
from azureml.core.webservice import Webservice
from azureml.core.image import ContainerImage

# configure the image
image_config = ContainerImage.image_configuration(execution_script="score.py", 
                                                  runtime="python", 
                                                  conda_file="myenv.yml")

service = Webservice.deploy_from_model(workspace=ws,
                                       name='sklearn-mnist-svc',
                                       deployment_config=aciconfig,
                                       models=[model],
                                       image_config=image_config)

service.wait_for_deployment(show_output=True)

Feitelijk ben je nu klaar. Het model is getraind, getest en draait nu in een container in Azure. Neem wederom een moment om te realiseren wat je feitelijk hebt gedaan en hoe lang je er mee bezig bent geweest. 

Uiteraard willen we ook de vruchten van ons werk kunnen plukken, dus willen we het endpoint van de webservice hebben:

In [ ]:
print(service.scoring_uri)

...en als we toch de URL hebben, moeten we deze ook even testen:

In [ ]:
import json

# find 30 random samples from test set
n = 30
sample_indices = np.random.permutation(X_test.shape[0])[0:n]

test_samples = json.dumps({"data": X_test[sample_indices].tolist()})
test_samples = bytes(test_samples, encoding='utf8')

# predict using the deployed model
result = service.run(input_data=test_samples)

# compare actual value vs. the predicted values:
i = 0
plt.figure(figsize = (20, 1))

for s in sample_indices:
    plt.subplot(1, n, i + 1)
    plt.axhline('')
    plt.axvline('')
    
    # use different color for misclassified sample
    font_color = 'red' if y_test[s] != result[i] else 'black'
    clr_map = plt.cm.gray if y_test[s] != result[i] else plt.cm.Greys
    
    plt.text(x=10, y =-10, s=result[i], fontsize=18, color=font_color)
    plt.imshow(X_test[s].reshape(28, 28), cmap=clr_map)
    
    i = i + 1
plt.show()

## Cleanup

De resources die we hebben opgespind zijn geen goedkope resources om continu in je Azure subscription te hebben staan. RUIM DEZE DUS OP. Je kan ze altijd weer opspinnen middels dit script. Eventueel kan je natuurlijk de ACI laten staan, zodat je wel tegen de webservice kan blijven aanpraten.

## Extra credit

Als je al je collega's waar mogelijk hebt geholpen en je als échte IT'er al die conversatie ook wel zat ben, dan kan je altijd nog een .net app bouwen die de zojuist gemaakte endpoint consumeert. Denk aan een console app welke een image post naar het endpoint, of wellicht ga je echt los en bouw je een UWP app met een canvas waar je cijfers op kan tekenen... of ben jij heel handig in Xamarin en Unity en blaas je ons allemaal weg met een implementatie tijdens een toekomstige R&D dag.

## Dank voor de aandacht
![Awesome](https://ih0.redbubble.net/image.45552731.0363/flat,550x550,075,f.jpg "You are awesome")